### 导入相关的库

In [1]:
import os
import json
import torch
import torch.nn.functional as F
import random
import time
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from sklearn.metrics import accuracy_score, f1_score, recall_score,precision_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier as SklearnMLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import numpy as np

### 将模型转移到gpu

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 导入Unixcoder模型

In [11]:
class ConbaLayer(nn.Module):
    def __init__(self, input_dim):
        super(ConbaLayer, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim)
        self.selective_fc = nn.Linear(input_dim, input_dim)
        self.swish = nn.SiLU()
        
        # 控制参数
        self.control_weight = nn.Parameter(torch.ones(input_dim))  # 可学习的权重参数
        self.feedback_weight = nn.Parameter(torch.ones(input_dim))  # 反馈控制权重
        
        # 状态空间参数
        self.A = nn.Linear(input_dim, input_dim, bias=False)  # 线性部分
        self.B = nn.Linear(input_dim, input_dim, bias=False)  # 非线性部分
        self.C = nn.Linear(input_dim, input_dim, bias=False)  # 输出映射

        self.fc2 = nn.Linear(input_dim, input_dim)
        
    def forward(self, x, previous_state=None):
        if previous_state is None:
            previous_state = torch.zeros_like(x, device=x.device)
        # 状态更新：使用状态空间模型计算当前状态
        state = self.A(previous_state) + self.B(x)
        
        # print(f'self.selective_fc(x):{self.selective_fc(x)}')
        # print(f'self.swish(self.selective_fc(x)):{self.swish(self.selective_fc(x))}')
        # 选择性激活和反馈控制
        selective_output = self.swish(self.selective_fc(x)) * x
        adjusted_output = selective_output * self.control_weight + state * self.feedback_weight
        print(f'self.control_weight:{self.control_weight}')
        print(f'self.feedback_weight:{self.feedback_weight}')
        
        # 输出层
        output = self.C(adjusted_output)
        output = self.fc2(output)
        return output, state  # 返回当前输出和更新后的状态


class SimCLRWithConba(nn.Module):
    def __init__(self, input_dim=768):
        super(SimCLRWithConba, self).__init__()
        self.conba_layer = ConbaLayer(input_dim)

    def forward(self, x, previous_state=None):
        # 传递状态信息到Conba层
        return self.conba_layer(x, previous_state)

contrastive_model = SimCLRWithConba(input_dim=768).to(device)
optimizer = optim.AdamW(contrastive_model.parameters(), lr=1e-4)

In [10]:
contrastive_model

SimCLRWithConba(
  (conba_layer): ConbaLayer(
    (fc1): Linear(in_features=768, out_features=768, bias=True)
    (selective_fc): Linear(in_features=768, out_features=768, bias=True)
    (swish): SiLU()
    (A): Linear(in_features=768, out_features=768, bias=False)
    (B): Linear(in_features=768, out_features=768, bias=False)
    (C): Linear(in_features=768, out_features=768, bias=False)
    (fc2): Linear(in_features=768, out_features=768, bias=True)
  )
)

In [12]:


# Sample input
x = torch.rand(1, 768).to(device)  # Single sample with dimension 768
previous_state = torch.rand(1,768).to(device)

# Forward pass through the model
print("Running the forward pass...")
output, state = contrastive_model(x, previous_state)


Running the forward pass...
self.control_weight:Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.